In [1]:
!sudo pip install -q transformers --upgrade

In [5]:
# new_model_name = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
# torch.cuda.empty_cache()

# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
# model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Load LLaMA tokenizer
from transformers import AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


# Load base model

model_name = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


In [ ]:
quantizer.quantize(
    input_data=dataset["train"],
    valid_data=dataset["validation"],
)